### Folder checks and import libraries

In [ ]:
#check where the notebook resides
os.getcwd()

In [ ]:
#check where the processed nbalogs reside - is it in same folder as notebook
os.listdir()

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import re
import collections
import datetime

pd.options.display.max_columns = 999

now = datetime.datetime.now()

#determine east and west teams (west auto excluded via list comprehension in working cells below)
ec = ['Atlanta', 'Boston', 'Brooklyn', 'Charlotte', 'Chicago','Cleveland','Detroit','Indiana',
      'Miami','Milwaukee','New York','Orlando','Philadelphia','Toronto','Washington']

### Scrap web and process file to nbalogs

In [2]:
#Define fixed variables
list_teams=['Boston Celtics','Brooklyn Nets','New York Knicks', 'Philadelphia 76ers', 'Toronto Raptors',
           'Chicago Bulls', 'Cleveland Cavaliers', 'Detroit Pistons', 'Indiana Pacers', 'Milwaukee Bucks',
           'Atlanta Hawks', 'Charlotte Hornets', 'Miami Heat', 'Orlando Magic', 'Washington Wizards',
           'Denver Nuggets', 'Minnesota Timberwolves', 'Oklahoma City Thunder', 'Portland Trail Blazers', 'Utah Jazz',
           'Golden State Warriors', 'Los Angeles Clippers', 'Los Angeles Lakers', 'Phoenix Suns', 'Sacramento Kings',
           'Dallas Mavericks', 'Houston Rockets', 'Memphis Grizzlies', 'New Orleans Pelicans', 'San Antonio Spurs']

dict_teams={'Boston Celtics': 'Boston',
 'Brooklyn Nets': 'Brooklyn',
 'New York Knicks': 'New York',
 'Philadelphia 76ers': 'Philadelphia',
 'Toronto Raptors': 'Toronto',
 'Chicago Bulls': 'Chicago',
 'Cleveland Cavaliers': 'Cleveland',
 'Detroit Pistons': 'Detroit',
 'Indiana Pacers': 'Indiana',
 'Milwaukee Bucks': 'Milwaukee',
 'Atlanta Hawks': 'Atlanta',
 'Charlotte Hornets': 'Charlotte',
 'Miami Heat': 'Miami',
 'Orlando Magic': 'Orlando',
 'Washington Wizards': 'Washington',
 'Denver Nuggets': 'Denver',
 'Minnesota Timberwolves': 'Minnesota',
 'Oklahoma City Thunder': 'Oklahoma City',
 'Portland Trail Blazers': 'Portland',
 'Utah Jazz': 'Utah',
 'Golden State Warriors': 'Golden State',
 'Los Angeles Clippers': 'LA Clippers',
 'Los Angeles Lakers': 'LA Lakers',
 'Phoenix Suns': 'Phoenix',
 'Sacramento Kings': 'Sacramento',
 'Dallas Mavericks': 'Dallas',
 'Houston Rockets': 'Houston',
 'Memphis Grizzlies': 'Memphis',
 'New Orleans Pelicans': 'New Orleans',
 'San Antonio Spurs': 'San Antonio'}

# years = [str(now.year -1), str(now.year-2), str(now.year-3), str(now.year-4)]
url_nba = 'https://www.oddsshark.com/nba/game-logs'

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
#Define function to count number of teams as sense-check (should be 30)
def count_teams(d):
    return sum([count_teams(v) if isinstance(v, dict) else 1 for v in d.values()])

In [4]:
#Define function to web scrap game logs (Take Note use below)
#Eg if I run From July to Mid Oct in year N during offseason, the results will be more season N-1/N
#(run during offseason - get data for previous season)

#Eg if I run from Mid Oct to Dec in year N of season, the results will be more season N/N+1
#Eg if I run from Jan to Jun in year N of season, the results will be more season N-1/N
#(run during season - get current data as season progress)

def read_nba_logs(url):
    urls_currentyr={}
    final_urls = collections.defaultdict(dict)
    
    html = urlopen(url, context=ctx).read()
    soup = BeautifulSoup(html, 'html.parser')

    tags = soup('a')

    for tag in tags:
        
        try:
            tag.contents[0]
        except:
            continue
        
        if tag.contents[0] in list_teams:
            team = tag.contents[0]
            base_url = re.sub(url,tag.get('href', None),url)
            urls_currentyr[team] = base_url
            final_urls[str(now.year)] = urls_currentyr
    
    final_urls = dict(final_urls)
    
    print('Total number of records: ', count_teams(final_urls))
    return final_urls

final_urls = read_nba_logs(url_nba)

Total number of records:  30


In [5]:
#check final_urls
final_urls

{'2023': {'Boston Celtics': 'https://www.oddsshark.com/stats/gamelog/basketball/nba/20722',
  'Brooklyn Nets': 'https://www.oddsshark.com/stats/gamelog/basketball/nba/20749',
  'New York Knicks': 'https://www.oddsshark.com/stats/gamelog/basketball/nba/20747',
  'Philadelphia 76ers': 'https://www.oddsshark.com/stats/gamelog/basketball/nba/20731',
  'Toronto Raptors': 'https://www.oddsshark.com/stats/gamelog/basketball/nba/20742',
  'Chicago Bulls': 'https://www.oddsshark.com/stats/gamelog/basketball/nba/20732',
  'Cleveland Cavaliers': 'https://www.oddsshark.com/stats/gamelog/basketball/nba/20735',
  'Detroit Pistons': 'https://www.oddsshark.com/stats/gamelog/basketball/nba/20743',
  'Indiana Pacers': 'https://www.oddsshark.com/stats/gamelog/basketball/nba/20737',
  'Milwaukee Bucks': 'https://www.oddsshark.com/stats/gamelog/basketball/nba/20725',
  'Atlanta Hawks': 'https://www.oddsshark.com/stats/gamelog/basketball/nba/20734',
  'Charlotte Hornets': 'https://www.oddsshark.com/stats/ga

In [6]:
#Define function to read into rawlogs dataframe
def process_to_rawlogs(url_dict):
    combineddf = pd.DataFrame()

    for nested_dict in url_dict.values():
        for team,url in nested_dict.items():
            scores = pd.read_html(url)[0]
            scores['Team'] = team
            cols = [scores.columns[-1]] + [col for col in scores if col != scores.columns[-1]]
            scores = scores[cols]
            combineddf = pd.concat([combineddf,scores])

    combineddf.index.rename('Played_Games',inplace=True)
    combineddf.reset_index(inplace=True)
    
    return combineddf

In [7]:
#check combineddf
combineddf = process_to_rawlogs(final_urls)
combineddf.tail()

,Played_Games,Team,Date,Opponent,Game,Result,Score,ATS,Spread,OU,Total
2455,77,San Antonio Spurs,"Apr 2, 2023",@ Sacramento,REG,NaN,NaN,NaN,NaN,NaN,NaN
2456,78,San Antonio Spurs,"Apr 4, 2023",@ Phoenix,REG,NaN,NaN,NaN,NaN,NaN,NaN
2457,79,San Antonio Spurs,"Apr 6, 2023",vs Portland,REG,NaN,NaN,NaN,NaN,NaN,NaN
2458,80,San Antonio Spurs,"Apr 8, 2023",vs Minnesota,REG,NaN,NaN,NaN,NaN,NaN,NaN
2459,81,San Antonio Spurs,"Apr 9, 2023",@ Dallas,REG,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#define 2regex functions for Score cleaning
def regex_win(string):
    str1 = re.search('\w+(?=-)',string)
    if str1:
        return int(str1.group())
    else:
        return np.nan
        
def regex_lose(string):
    str2 = re.search('(?<=-)\w+',string)
    if str2:
        return int(str2.group())
    else:
        return np.nan

In [9]:
#Define function to process rawlogs to nbalogs dataframe
def process_to_nbalogs(rawlogs_df):
    
    #change date column to datetime
    rawlogs_df['Date']=pd.to_datetime(rawlogs_df['Date'])

    #identify home and away
    rawlogs_df['Home_Away'] = np.where(rawlogs_df.Opponent.str.match('vs'),'Home','Away')

    #regex to split out opponents
    rawlogs_df['Opponent_clean'] = rawlogs_df['Opponent'].apply(lambda x:re.findall('(?<=@\s).*|(?<=vs\s).*',x)[0])

    rawlogs_df['Score']=rawlogs_df.Score.astype(str)

    #regex to split out Wpoints and Losepoints
    rawlogs_df['Wpoints'] = rawlogs_df.Score.apply(regex_win)
    rawlogs_df['Lpoints'] = rawlogs_df.Score.apply(regex_lose)

    #dealing with ATS
    rawlogs_df['ATS_margin_W'] = rawlogs_df[['Wpoints','Spread']].sum(axis=1).where(rawlogs_df['Result']=='W')
    rawlogs_df['ATS_margin_L'] = rawlogs_df[['Lpoints','Spread']].sum(axis=1).where(rawlogs_df['Result']=='L')
    rawlogs_df['ATS_combi'] = rawlogs_df['ATS_margin_W'].fillna(rawlogs_df['ATS_margin_L'])

    #process ATS
    rawlogs_df['minusL'] = -1 * rawlogs_df['Lpoints']
    rawlogs_df['minusATScombi'] = -1 * rawlogs_df['ATS_combi']
    rawlogs_df['ATS_clear_W'] = rawlogs_df[['ATS_combi','minusL']].sum(axis=1).where(rawlogs_df['Result']=='W')
    rawlogs_df['ATS_clear_L'] = rawlogs_df[['Wpoints','minusATScombi']].sum(axis=1).where(rawlogs_df['Result']=='L')
    rawlogs_df['ATS_clear_L'] = -1* rawlogs_df['ATS_clear_L']
    rawlogs_df['ATS_final'] = rawlogs_df['ATS_clear_W'].fillna(rawlogs_df['ATS_clear_L'])

    rawlogs_df.drop(['ATS_margin_W', 'ATS_margin_L', 'ATS_combi', 'minusL', 'minusATScombi', 
                   'ATS_clear_W', 'ATS_clear_L'], axis=1,inplace=True)

    rawlogs_df = rawlogs_df.replace('nan',np.nan)
    
    #process OU
    rawlogs_df.rename(columns={'Total': "Total_HC"},inplace=True)
    rawlogs_df['Total_actual'] = rawlogs_df['Wpoints'] + rawlogs_df['Lpoints']
    
    rawlogs_df['minus_OU'] = -1 * rawlogs_df['Total_HC']
    rawlogs_df['O_margin'] = rawlogs_df[['Total_actual','minus_OU']].sum(axis=1).where(rawlogs_df['OU']=='O')
    rawlogs_df['U_margin'] = rawlogs_df[['Total_actual','minus_OU']].sum(axis=1).where(rawlogs_df['OU']=='U')
    rawlogs_df['P_margin'] = rawlogs_df[['Total_actual','minus_OU']].sum(axis=1).where(rawlogs_df['OU']=='P')
    rawlogs_df['OU_final'] = rawlogs_df['O_margin'].fillna(rawlogs_df['U_margin'])
    rawlogs_df['OU_final'] = rawlogs_df['OU_final'].fillna(rawlogs_df['P_margin'])

    rawlogs_df.drop(['O_margin', 'U_margin', 'P_margin','minus_OU'], axis=1,inplace=True)
    
    #drop Playoff games
    rawlogs_df = rawlogs_df.loc[rawlogs_df.Game == 'REG']
    
    #further processing
    rawlogs_df = rawlogs_df[['Played_Games', 'Team', 'Date', 'Game', 'Result', 'Score', 'ATS', 
                   'Spread', 'OU', 'Total_HC', 'Home_Away', 'Opponent_clean', 'Wpoints', 'Lpoints', 
                   'Total_actual','ATS_final', 'OU_final']]
    
    rawlogs_df = rawlogs_df[~rawlogs_df.Result.isna()]
    
    rawlogs_df['Team'] = rawlogs_df['Team'].map(dict_teams)
    
    rawlogs_df.reset_index(drop=True,inplace=True)
    
    return rawlogs_df

In [10]:
#check processed nbalogs
nbalogs = process_to_nbalogs(combineddf)
nbalogs.head()

,Played_Games,Team,Date,Game,Result,Score,ATS,Spread,OU,Total_HC,Home_Away,Opponent_clean,Wpoints,Lpoints,Total_actual,ATS_final,OU_final
0,0,Boston,2022-10-18,REG,W,126-117,W,-3.0,O,216.5,Home,Philadelphia,126.0,117.0,243.0,6.0,26.5
1,1,Boston,2022-10-21,REG,W,111-104,W,-2.0,U,221.0,Away,Miami,111.0,104.0,215.0,5.0,-6.0
2,2,Boston,2022-10-22,REG,W,126-120,L,-8.5,O,215.5,Away,Orlando,126.0,120.0,246.0,-2.5,30.5
3,3,Boston,2022-10-24,REG,L,120-102,L,-5.5,U,226.0,Away,Chicago,120.0,102.0,222.0,-23.5,-4.0
4,4,Boston,2022-10-28,REG,L,132-123,L,-7.0,O,218.5,Home,Cleveland,132.0,123.0,255.0,-16.0,36.5


In [11]:
#export to csv
nbalogs.to_csv(f'nbalogs_{str(now.date())}.csv')